We need to first download all the dependencies we need to explore the data

In [2]:
import numpy as np
import pandas as pd
import json
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
import io

Load the data into Pandas

In [3]:
url = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")


Make the dataframe

In [4]:
toronto_neigh = pd.DataFrame(url[0])
toronto_neigh

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Eliminate "Not assigned" values

In [5]:
toronto_neigh = toronto_neigh[toronto_neigh.Borough !='Not assigned']

In [6]:
toronto_neigh.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Merge rows with the same Postal Code

In [7]:
toronto_neigh.set_index(['Postal Code','Borough'],inplace=True)


In [8]:
merge_df = toronto_neigh.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)

In [9]:
merge_df.head(10)

,,Neighbourhood
Postal Code,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


Set the index

In [10]:
reset_df = merge_df.reset_index()


In [11]:
df = pd.DataFrame(reset_df)

In [12]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Print number of rows

In [13]:
df.shape

(103, 3)

Load Geospatial_Coordinates.csv file into pandas

In [14]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_df7f0156843749d1a4ffe5269f33f88c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='q3MrqrBtJ_3wnjfnXuFpe1XHwQUHLlvw92g3XxyBewBy',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_df7f0156843749d1a4ffe5269f33f88c.get_object(Bucket='capstoneproject-donotdelete-pr-sifh5366on2bx8',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_geo_coor = pd.read_csv(body)
df_geo_coor.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two dataframes into one dataframe

In [15]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df_toronto.head(12)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Remove 'Postal Code' column

In [16]:
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head(12)

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,North York,Don Mills,43.745906,-79.352188
8,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Create a map of Toronto

In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [18]:
map_toronto = folium.Map(Location = [latitude, longitude], zoom_start = 10)

#Add markers to the map

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [19]:

CLIENT_ID = 'FVQJB0SVJXUHEBUROHDVXTYC4GXGKDGD4JGVBKZG2XMRNA5L'
CLIENT_SECRET = 'RAU3ZF1WX0SIEQO2KOMSFTBEAYEV5PL0MGPOX022HQG0QFPV'
VERSION = '20210302'
radius=500
print('Client ID:', CLIENT_ID)
print ('Clent Secret:', CLIENT_SECRET)

Client ID: FVQJB0SVJXUHEBUROHDVXTYC4GXGKDGD4JGVBKZG2XMRNA5L
Clent Secret: RAU3ZF1WX0SIEQO2KOMSFTBEAYEV5PL0MGPOX022HQG0QFPV


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Explore Neighbordhoods in Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=20):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe called toronto_venues

In [23]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                 latitudes=df_toronto['Latitude'],
                                 longitudes=df_toronto['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Check the size of the dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(1075, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Find out how many unique catergories can be extracted from the venues

In [25]:
print('There are {} uniques categories.'.format(toronto_venues['Venue Category'].nunique()))

There are 213 uniques categories.


Analyze each neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(10)


,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
toronto_grouped.shape

(96, 213)

Print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue_category ','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
             venue_category   freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2         Chinese Restaurant  0.25
3             Breakfast Spot  0.25
4              Movie Theater  0.00


----Alderwood, Long Branch----
  venue_category   freq
0     Pizza Place  0.25
1             Gym  0.12
2    Skating Rink  0.12
3        Pharmacy  0.12
4             Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
       venue_category   freq
0                 Bank  0.10
1          Coffee Shop  0.10
2        Deli / Bodega  0.05
3     Sushi Restaurant  0.05
4  Fried Chicken Joint  0.05


----Bayview Village----
       venue_category   freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
      venue_category   freq
0  Italian Restaurant  0.10
1      Sandwich Place  0.10
2         Coffee Shop  0.10
3      

       venue_category   freq
0           Playground  0.33
1                 Park  0.33
2         Intersection  0.33
3                Motel  0.00
4  Martial Arts School  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
          venue_category   freq
0                     Gym  0.07
1  Thrift / Vintage Store  0.07
2           Grocery Store  0.07
3              Kids Store  0.07
4    Fast Food Restaurant  0.07


----Moore Park, Summerhill East----
              venue_category   freq
0                Tennis Court   1.0
1                 Yoga Studio   0.0
2  Modern European Restaurant   0.0
3                      Lounge   0.0
4         Martial Arts School   0.0


----New Toronto, Mimico South, Humber Bay Shores----
  venue_category   freq
0             Gym  0.09
1          Bakery  0.09
2        Pharmacy  0.09
3       Pet Store  0.09
4      Restaurant  0.09


----North Park, Maple Leaf Park, Upwood Park----
              venue_categ

Put that into a pandas dataframe

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Chinese Restaurant,Breakfast Spot,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Pub,Sandwich Place,Curling Ice,Distribution Center,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Grocery Store,Sandwich Place,Ice Cream Shop,Diner,Middle Eastern Restaurant,Deli / Bodega,Mobile Phone Shop,Bridal Shop
3,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Pub,Thai Restaurant,Butcher,Café,Grocery Store
5,Berczy Park,Coffee Shop,Farmers Market,Seafood Restaurant,Park,Beer Bar,Cocktail Bar,Jazz Club,Basketball Stadium,Thai Restaurant,Fountain
6,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Women's Store,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store
7,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,Burrito Place
8,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Skate Park,Light Rail Station
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Rental Car Location,Plane,Harbor / Marina,Coffee Shop,Bar,Airport Gate


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [42]:
# best number of k is 5
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 1, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,0,Park,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,0,Park,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
5,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
6,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
7,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
8,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
9,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center


Visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Cluster 1

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Neighborhood Longitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.753259,-79.329656,-79.332140,Park,0,Park,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
1,43.753259,-79.329656,-79.333114,Food & Drink Shop,0,Park,Food & Drink Shop,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
191,43.689026,-79.453512,-79.456300,Park,0,Park,Women's Store,Pool,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
192,43.689026,-79.453512,-79.456333,Women's Store,0,Park,Women's Store,Pool,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
193,43.689026,-79.453512,-79.448922,Pool,0,Park,Women's Store,Pool,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
194,43.689026,-79.453512,-79.448924,Park,0,Park,Women's Store,Pool,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
373,43.685347,-79.338106,-79.335007,Park,0,Pizza Place,Park,Convenience Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
374,43.685347,-79.338106,-79.335007,Convenience Store,0,Pizza Place,Park,Convenience Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
375,43.685347,-79.338106,-79.333846,Pizza Place,0,Pizza Place,Park,Convenience Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
632,43.728020,-79.388790,-79.394382,Park,0,Bus Line,Park,Swim School,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


Cluster 2

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Neighborhood Longitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,43.725882,-79.315572,-79.315635,Hockey Arena,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
3,43.725882,-79.315572,-79.312785,Portuguese Restaurant,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
4,43.725882,-79.315572,-79.313103,Coffee Shop,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
5,43.725882,-79.315572,-79.312860,Pizza Place,1,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
63,43.706397,-79.309937,-79.312913,Gastropub,1,Pizza Place,Pharmacy,Café,Breakfast Spot,Bank,Flea Market,Intersection,Athletics & Sports,Gastropub,Gym / Fitness Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,43.602414,-79.543484,-79.545048,Coffee Shop,1,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Pub,Sandwich Place,Curling Ice,Distribution Center,Discount Store
974,43.602414,-79.543484,-79.547252,Pizza Place,1,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Pub,Sandwich Place,Curling Ice,Distribution Center,Discount Store
975,43.602414,-79.543484,-79.544395,Sandwich Place,1,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Pub,Sandwich Place,Curling Ice,Distribution Center,Discount Store
976,43.602414,-79.543484,-79.545694,Pharmacy,1,Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Pub,Sandwich Place,Curling Ice,Distribution Center,Discount Store


Cluster 3

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Neighborhood Longitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
552,43.789053,-79.408493,-79.410022,Park,2,Park,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
642,43.706876,-79.518188,-79.521705,Park,2,Park,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
643,43.706876,-79.518188,-79.522648,Park,2,Park,Women's Store,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


Cluster 4

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Neighborhood Longitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
856,43.689574,-79.38316,-79.3829,Tennis Court,3,Tennis Court,Women's Store,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


Cluster 5

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Neighborhood Longitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,43.654260,-79.360636,-79.362017,Bakery,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
7,43.654260,-79.360636,-79.361809,Coffee Shop,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
8,43.654260,-79.360636,-79.358008,Distribution Center,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
9,43.654260,-79.360636,-79.359874,Spa,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
10,43.654260,-79.360636,-79.356980,Restaurant,4,Coffee Shop,Park,Breakfast Spot,Bakery,Historic Site,Spa,Farmers Market,Restaurant,Pub,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,43.628841,-79.520999,-79.518041,Fast Food Restaurant,4,Grocery Store,Fast Food Restaurant,Tanning Salon,Supplement Shop,Bakery,Discount Store,Thrift / Vintage Store,Convenience Store,Hardware Store,Sandwich Place
1071,43.628841,-79.520999,-79.519006,Tanning Salon,4,Grocery Store,Fast Food Restaurant,Tanning Salon,Supplement Shop,Bakery,Discount Store,Thrift / Vintage Store,Convenience Store,Hardware Store,Sandwich Place
1072,43.628841,-79.520999,-79.518290,Kids Store,4,Grocery Store,Fast Food Restaurant,Tanning Salon,Supplement Shop,Bakery,Discount Store,Thrift / Vintage Store,Convenience Store,Hardware Store,Sandwich Place
1073,43.628841,-79.520999,-79.518238,Thrift / Vintage Store,4,Grocery Store,Fast Food Restaurant,Tanning Salon,Supplement Shop,Bakery,Discount Store,Thrift / Vintage Store,Convenience Store,Hardware Store,Sandwich Place
